In [6]:
!pip install opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 30.6 MB 26 kB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [7]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import itertools
import time
import zipfile
import torch
import torchvision
import gc
import pickle
from sklearn import svm
from skimage import color
from skimage import io
from torch.utils.data import Dataset, DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import time
import warnings

print(cv2.__version__)

3.4.2


In [8]:
cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [9]:
class_names = [name[13:] for name in glob.glob('./data/train/*')]
class_names = dict(zip(range(len(class_names)), class_names))
print("class_names: %s " % class_names)
n_train_samples_per_class = 150
n_test_samples_per_class = 50

def load_dataset(path, num_per_class=-1):
    data = []
    labels = []
    for id, class_name in class_names.items():
        print("Loading images from class: %s" % id)
        img_path_class = glob.glob(path + class_name + '/*.jpg')
        if num_per_class > 0:
            img_path_class = img_path_class[:num_per_class]
        labels.extend([id]*len(img_path_class))
        for filename in img_path_class:
            data.append(cv2.imread(filename, 0))
    return data, labels

# load training dataset
#train_data, train_label = load_dataset('./data/train/')
train_data, train_label = load_dataset('./data/train/', n_train_samples_per_class)
n_train = len(train_label)
print("n_train: %s" % n_train)

# load testing dataset
#test_data, test_label = load_dataset('./data/test/')
test_data, test_label = load_dataset('./data/test/', n_test_samples_per_class)
n_test = len(test_label)
print("n_test: %s" % n_test)

class_names: {0: 'Forest', 1: 'Industrial', 2: 'Flower', 3: 'Coast', 4: 'InsideCity', 5: 'Office', 6: 'Bedroom', 7: 'Highway', 8: 'Street', 9: 'TallBuilding', 10: 'LivingRoom', 11: 'Suburb', 12: 'OpenCountry', 13: 'Mountain', 14: 'Kitchen', 15: 'Store'} 
Loading images from class: 0
Loading images from class: 1
Loading images from class: 2
Loading images from class: 3
Loading images from class: 4
Loading images from class: 5
Loading images from class: 6
Loading images from class: 7
Loading images from class: 8
Loading images from class: 9
Loading images from class: 10
Loading images from class: 11
Loading images from class: 12
Loading images from class: 13
Loading images from class: 14
Loading images from class: 15
n_train: 2400
Loading images from class: 0
Loading images from class: 1
Loading images from class: 2
Loading images from class: 3
Loading images from class: 4
Loading images from class: 5
Loading images from class: 6
Loading images from class: 7
Loading images from class: 8


In [10]:
# Save intermediate image data into disk
file = open('train.pkl','wb')
pickle.dump(train_data, file)
pickle.dump(train_label, file)
file.close()

file = open('test.pkl','wb')
pickle.dump(test_data, file)
pickle.dump(test_label, file)
file.close()

In [11]:
# Load intermediate image data from disk
file = open('train.pkl', 'rb')
train_data = pickle.load(file)
train_label = pickle.load(file)
file.close()

file = open('test.pkl', 'rb')
test_data = pickle.load(file)
test_label = pickle.load(file)
file.close()

print(len(train_data), len(train_label)) # Verify number of training samples
print(len(test_data), len(test_label))   # Verify number of testing samples

2400 2400
400 400


In [12]:
#plt.imshow(train_data[1], cmap='gray') # Verify image
img_new_size = (240, 240)

train_data = list(map(lambda x: cv2.resize(x, img_new_size), train_data))
#print('old',train_data)
train_data = np.stack(train_data)
#print('new',train_data)
train_label = np.array(train_label)
#print('new',train_label)

test_data = list(map(lambda x: cv2.resize(x, img_new_size), test_data))
test_data = np.stack(test_data)
test_label = np.array(test_label)
#plt.imshow(train_data[1], cmap='gray') # Verify image


In [13]:
import gc
gc.collect()

103

In [14]:
train_data_2 = train_data.copy()
test_data2 = test_data.copy()
train_label2 = train_label.copy()
label2 = test_label.copy()
no_of_clusters = 50


In [15]:
def clustering(image_list):
  desc_list = []
  for img in image_list:
    extractor = cv2.xfeatures2d.SIFT_create()
    kp = extractor.detect(img)
    dkp = [cv2.KeyPoint(k.pt[0], k.pt[1],k.size) for k in kp]
    keyp, descriptors = extractor.compute(img, dkp)
    desc_list.append(descriptors)
    
  desc_list=np.asarray(desc_list)
  desc_list=np.concatenate(desc_list, axis=0)
  #desc_list = np.sqrt(desc_list).astype(int)

  
  kmeans = MiniBatchKMeans(n_clusters=50, random_state=0).fit(desc_list)
  
  return kmeans
  

In [16]:
def histogram(image_list,kmeans):
  features = []
  for img in image_list:
    extractor = cv2.xfeatures2d.SIFT_create()
    ###
    kp = extractor.detect(img)
    dkp = [cv2.KeyPoint(k.pt[0], k.pt[1],k.size) for k in kp]
    keyp, descriptors = extractor.compute(img, dkp)
    ###
    #kp, descriptors = extractor.detectAndCompute(img,None)
    predict_class = kmeans.predict(descriptors)
    hist, bin_edges=np.histogram(predict_class, bins=50)
    
    features.append(hist)
    
  features=np.asarray(features)
  return features

In [17]:
start = time.process_time()
kmeans = clustering(train_data_2)
time7 = time.process_time() - start
print("Time Consumption for Clustering Features: ",time7 , "seconds")


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Time Consumption for Clustering Features:  338.364721468 seconds


In [18]:
start = time.process_time()
train_feat = histogram(train_data_2,kmeans)
test_feat = histogram(test_data2,kmeans)
time8 = time.process_time() - start
print("Time Consumption for Histogram Generation: ",time8 , "seconds")


Time Consumption for Histogram Generation:  400.881764942 seconds


In [19]:
start = time.process_time()
classifier = KNeighborsClassifier(n_neighbors=10, metric = 'minkowski', p=2)
classifier.fit(train_feat, train_label2)
time3 = time.process_time() - start

print("Time Consumption in Training (Bag of SIFT Representation + Nearest Neighbor Classifer) : ",time3 , "seconds")

start = time.process_time()
pred2 = classifier.predict(test_feat)
time4 = time.process_time() - start
print("The Prediction accuracy of this model is {:.2f}%".format(accuracy_score(label2,pred2)*100)) 

print("Time Consumption in Prediction (Bag of SIFT Representation + Nearest Neighbor Classifer) : ",time4 , "seconds")


Time Consumption in Training (Bag of SIFT Representation + Nearest Neighbor Classifer) :  0.015419666999946458 seconds
The Prediction accuracy of this model is 44.00%
Time Consumption in Prediction (Bag of SIFT Representation + Nearest Neighbor Classifer) :  0.12154604700003802 seconds


In [20]:
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#Standardizing the histogram matrix
train_featn = scaler.fit_transform(train_feat)
test_featn = scaler.fit_transform(test_feat)

label3 = test_label.copy()

start = time.process_time()
clf = svm.LinearSVC(random_state=0, C = 4.6)    #Using lambda = 4.6 
clf.fit(train_featn, train_label2) 
time5 = time.process_time() - start
print("Time Consumption in Training (Bag of SIFT Representation + one-vs-all SVMs) : ",time5 , "seconds")

start = time.process_time()
pred3 = clf.predict(test_featn)
time6 = time.process_time() - start
print("Using Lambda = 4.6")
print("The Prediction accuracy of this model is {:.2f}%".format(accuracy_score(label3,pred3)*100)) 
print("Time Consumption in Prediction (Bag of SIFT Representation + one-vs-all SVMs) : ",time6 , "seconds")


Time Consumption in Training (Bag of SIFT Representation + one-vs-all SVMs) :  3.6915149569999812 seconds
Using Lambda = 4.6
The Prediction accuracy of this model is 47.50%
Time Consumption in Prediction (Bag of SIFT Representation + one-vs-all SVMs) :  0.0013282639999943058 seconds
